In [ ]:
!pip install fake_useragent
from bs4 import BeautifulSoup
from bs4.element import Tag
import requests
import time
from fake_useragent import UserAgent
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error

Парсинг устроен в несколько этапов:

1. Функция `get_information`, главная цель которой - делать обращение на страницу сайта и собирать оттуда всю информацию. Эта часть выполнена с помощью BeautifulSoup

In [ ]:
def get_information(url):
  response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
  tree = BeautifulSoup(response.content, 'html.parser')
  return tree

2. Вручную были выписаны те предметы, цены репетиторов на которые мы хотели бы предсказывать. Выбор неслучаен - были отобраны наиболее популярные предметы, где количество доступных для выбора репетиторов было более, чем 95 человек.

In [ ]:
object_list = {'Русский язык': 'russkij-yazyk',
               'Математика': 'matematika-elementarnaya',
               'Английский язык': 'anglijskij-yazyk',
               'Биология': 'biologiya',
               'Обществознание': 'obshchestvoznanie',
               'Музыка': 'muzyka',
               'География': 'geografiya',
               'Информатика': 'informatika',
               'Испанский язык': 'ispanskij-yazyk',
               'Логопед' : 'logopedy',
               'Физика' : 'fizika',
               'Химия' : 'himiya',
               'Биология' : 'biologiya',
               'Экономика' : 'ekonomika'
               }

3. Функция get_href - получение ссылок на страницу преподавателя. Мы проходимся по всем предметам из object_list и для каждого предмета считываем первые 6 страниц с преподавателями, после чего создаем DataFrame all_hrefсодержащий ссылку на личную страницу преподавателя в первом столбце, а во втором - название предмета, по которому он оказывает услуги.

In [ ]:
def get_href():
  for key in object_list.keys():
    for i in range(1,7):
      url = f'https://repetitor.ru/repetitors/{object_list[key]}?page={i}&per-page=15'
      tree = get_information(url)
      tutors_raw = tree.body.find_all('div', {'class' : 'tutors__main'})
      tutors_cards = tutors_raw[0].find_all('li', {'class' : 'tutor-card'})
      cur_href = [item.div.a.get('href') for item in tutors_cards]
      cur_object = [key for i in range(len(cur_href))]
      all_href['href'].extend(cur_href)
      all_href['object_name'].extend(cur_object)
      time.sleep(0.4)

In [ ]:
all_href = {'href' : [], 'object_name' : []}
get_href()

In [ ]:
df_href = pd.DataFrame(all_href)

In [ ]:
print(df_href.shape)
df_href

4. Ну и вишенка на торте нашего парсинга - функция get_tutor. Эта функция проходится по каждой ссылке из массива all_href, заходя на личную страницу преподавателя, собирая нужную нам информацию.

In [ ]:
from tqdm import tqdm

def get_tutor():
  for i in tqdm(range(df_href.shape[0])):
    url, object_name =  df_href.iloc[i]
    tree = get_information(f'https://repetitor.ru{url}')

    #страница! препода
    try:
      tutor_raw_inf = tree.body.find_all('div', {'class' : 'tutor__wrap'})[0]
    except:
      print(f'Объебос на странице {url}')
      continue

    #определяю предмет препода
    dict_raw['Предмет'].append(object_name)

    #сначала определяю, подтверждена ли личность преподавателя (True - подтверждена; False - нет)
    tutor_person_confirmed_raw = tutor_raw_inf.find('div', {'class' : 'tutor-badge-blue'})
    if type(tutor_person_confirmed_raw) == Tag:
      tutor_person_confirmed = True
    else:
      tutor_person_confirmed = False
    dict_raw['Личность подтверждена'].append(tutor_person_confirmed)

    #здесь я определяю стаж и статус
    try:
      status_staj_raw = tutor_raw_inf.find('p', {'class' : "tutor-head__meta"}).text.strip().split('\n')
      dict_raw['Статус'].append(status_staj_raw[0].split(', ')[0])
      dict_raw['Стаж'].append(status_staj_raw[-1].split()[0])
    except:
      dict_raw['Статус'].append(None)
      dict_raw['Стаж'].append(None)

    #определяю кол-во времени репета на сайте и кол-во заказов в работе
    try:
      onsite_orders_raw = tutor_raw_inf.find('div', {'class' : "tutor-head__activity"}).text.strip().split('/')
      onsite = onsite_orders_raw[0].split()[-2:]
      if onsite[0] == 'Меньше':
        real_onsite = 0
      else:
        real_onsite = int(onsite[0])/12 if onsite[1] in ['месяца', 'месяцев'] else onsite[0]
    except:
      real_onsite = None

    try:
      orders = onsite_orders_raw[1].strip().split()[0]
    except:
      orders = None

    dict_raw['На сайте'].append(real_onsite)
    dict_raw['Заказы в работе'].append(orders)

    #здесь я определяю оценку
    try:
      mark = tutor_raw_inf.find('div', {'class' : "tutor-head__rating-count"}).text.strip()
    except:
      mark = None
    dict_raw['Оценка'].append(mark)

    #здесь я определяю количество отзывов
    try:
      number_of_review = tutor_raw_inf.find('div', {'class' : "tutor-head__rating"}).a.text.strip().split()[0]
    except:
      number_of_review = None
    dict_raw['Количество отзывов'].append(number_of_review)

    #здесь я определяю формат занятий - дистанционно али очно
    try:
      format_of_study = tutor_raw_inf.find('div', {'class' : "tutor-format__switches tabs__switches"}).button.text
    except:
      format_of_study = None
    dict_raw['Формат обучения'].append(format_of_study)

    #здесь я определяю среднюю цену
    try:
      prices_raw = tutor_raw_inf.find('div', {'class' : "tutor-format__prices"})
      prices = prices_raw.find_all('td')[1::2]
      price = 0
      for price_elem in prices:
        price_currency, time_minute = str(price_elem).strip('</td>').split('/')
        price_without_currency = price_currency.split()[0]
        time_hour = int(time_minute.split()[0])/60
        price += float(price_without_currency) * time_hour
      price /= len(prices)
    except:
      price = None
    dict_raw['Цена'].append(price)

    #образование тьютора
    excellence = False
    try:
      tutor_education = tutor_raw_inf.find('table', {'class' : "tutor-education__table"}).span.text.strip()
      if 'С отличием' in tutor_education:
        excellence = True
        tutor_education = tutor_education[:tutor_education.rfind('С отличием')-2]
      elif 'с отличием' in tutor_education:
        excellence = True
        tutor_education = tutor_education[:tutor_education.rfind('с отличием')-2]
    except:
      tutor_education = None
    dict_raw['Образование'].append(tutor_education)

    #с отличием ли окончил?
    dict_raw['С отличием'].append(excellence)

    time.sleep(0.1)

In [ ]:
dict_raw = {
    'Личность подтверждена' : [],
    'Статус' : [],
    'Стаж' : [],
    'На сайте' : [],
    'Заказы в работе' : [],
    'Оценка' : [],
    'Количество отзывов' : [],
    'Формат обучения' : [],
    'Образование' : [],
    'Предмет' : [],
    'Цена' : [],
    'С отличием' : []
}

In [ ]:
get_tutor()
df_raw = pd.DataFrame(dict_raw)
df_raw

**Информация о полученных данных (по столбцам)**

1. Личность подтверждена - бинарный признак со значением `{0,1}`. Данные о том, предоставил ли репетитор свои паспортные данные платформе.

2. Статус - категориальный признак, область значения которого следующая: `'Частный преподаватель', 'Школьный преподаватель' 'Преподаватель университета или колледжа', 'Аспирант или ординатор очной формы обучения', 'Студент'`.некоторая текущая "должность" преподавателя.

3. Стаж - числовой признак, дающий понимание о том, какое количество лет преподаватель обучает своему предмету. Область значения признака - `R+`.

4. На сайте - числовой признак, отображающий количество времени, прошедшего с регистрации тьютора на сайте repetitor.ru. Область значения признака - `R+`.

5. Заказы в работе - числовой признак, иллюстрирующий какое количество учеников у конкретного преподавателя было на момент парсинга 01.06.2024. Область значения признака - `Z`.

6. Оценка - числовой признак, отображающий среднюю оценку по отзывам учеников. Область значения признака - `[0,5]`.

7. Количество отзывов - числовой признак, количество оставленных учениками отзывов о преподавателе. Область значения признака - `Z`.

8. Формат обучения - категорильный признак, предлогаемый репетитором формат занятий, область значения - `{Дистанционно, У ученика, У преподавателя}`.

9. Образование - категориальный признак, ВУЗ - который закончил преподаватель. Область значения - название ВУЗов России и мира.

10. Предмет - категориальный признак, предмет, который преподает тьютор. Область значений - ключи словаря `object_list` со 2 шага парсинга

11. Цена - таргет, который мы собираемся предсказывать. Область значения - `R+`

12. С отличием - бинарный признак, отображающий, был ли закончен ВУЗ преподавателем с отличием или нет.

In [ ]:
# все запаршенные данные записываю в файл
df_raw.to_csv('parsing.csv', encoding='utf-8')